In [1]:
import numpy as np
import pandas as pd


In [2]:
def importData(path = 'tennis.csv'):

    data = pd.read_csv(path, header =0 , skiprows = 0)
    print(data.head())
    data.target  =data['Play']
    
    return data



In [3]:
def entropy(data):

    d = data.iloc[:,-1]
    d= d.value_counts()
    s = 0
    for v in d.keys():
        p = d[v]/sum(d)
        s-=p*np.log2(p)
    return s



In [4]:
def values(attr):
    l = []
    for x in attr:
        if x not in l:
            l.append(x)
    return l



In [5]:
def IG(data, A):

    Es = entropy(data)
    val = values(data[A])
    s_c = data[A].value_counts()
    s_v = []

    for v in range(len(val)):
        ds = data[data[A] == val[v]]
        s = 0
        for res in values(data.iloc[:,-1]):
            try:
                pi = ds.iloc[:,-1].value_counts()[res]/len(ds)
                s -= pi*np.log2(pi)
            except:
                s = 0
        s_v.append(s)
    for i in range(len(val)):
        Es = Es - s_c[val[i]]*s_v[i]/sum(s_c)
        
    return Es

    

In [6]:
class Node():

    def __init__(self, name = None, attr = None):
        self.name = name
        self.attr = attr
    def call_(self):
        return self.name

def DTNode(data, features_used):
    node = Node()
    IGmax = 0
    vbest = None
    val_list = [ v for v in values(data)[:-1] if v not in features_used]

    if val_list != []:

        for v in val_list:

            if IG(data, v) > IGmax:
                IGmax = IG(data, v)
                v_best = v
        if v_best:
            features_used.append(v_best)
            node.name = v_best
            node.attr = values(data[v_best])
            return node
        else:
            return None
    return None

def DTC(data, features_used):
    root = DTNode(data, features_used)
    DT_dict = {}
    if root != None:
        item = []
        for attr in root.attr:
            dataN = data[data[root.name] == attr]
            if entropy(dataN) == 0:
                item.append((attr, values(dataN.iloc[:, -1])[0]))
            else:
                dt = DTC(dataN, features_used)
                item.append((attr, dt))
        DT_dict[root.name] = item
    return DT_dict

In [7]:
# Load the dataset
data = importData('tennis.csv')

# Initialize the list to keep track of features used
features_used = []

# Build the decision tree using the ID3 algorithm
decision_tree = DTC(data, features_used)

# Print or visualize the decision tree
print(decision_tree)

    Outlook Temperature Humidity    Wind Play
0     Sunny         Hot     High    Weak   No
1     Sunny         Hot     High  Strong   No
2  Overcast         Hot     High    Weak  Yes
3      Rain        Mild     High    Weak  Yes
4      Rain        Cool   Normal    Weak  Yes
{'Outlook': [('Sunny', {'Humidity': [('High', 'No'), ('Normal', 'Yes')]}), ('Overcast', 'Yes'), ('Rain', {'Wind': [('Weak', 'Yes'), ('Strong', 'No')]})]}


C:\Users\GOD WORLD\AppData\Local\Temp\ipykernel_18540\3160606400.py:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  data.target  =data['Play']


In [8]:
import numpy as np
import pandas as pd

def importData(path='tennis.csv'):
    data = pd.read_csv(path)
    return data

def gini_index(groups, classes):
    n_instances = float(sum([len(group) for group in groups]))
    gini = 0.0
    for group in groups:
        size = float(len(group))
        if size == 0:
            continue
        score = 0.0
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
        gini += (1.0 - score) * (size / n_instances)
    return gini

def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

def get_split(dataset):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(len(dataset[0])-1):
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)

def build_tree(train, max_depth, min_size):
    root = get_split(train)
    split(root, max_depth, min_size, 1)
    return root

def print_tree(node, depth=0):
    if isinstance(node, dict):
        if isinstance(node['value'], str):
            print('%s[X%d = %s]' % ((depth*' ', (node['index']+1), node['value'])))
        else:
            print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), float(node['value']))))
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)
    else:
        print('%s[%s]' % ((depth*' ', node)))

# Load the dataset
data = importData('tennis.csv')
dataset = data.values.tolist()

# Define the maximum depth of the tree and the minimum number of samples required to split a node
max_depth = 3
min_size = 2

# Build the CART decision tree
tree = build_tree(dataset, max_depth, min_size)

# Print the constructed decision tree
print_tree(tree)


[X1 = Rain]
 [X1 = Overcast]
  [Yes]
  [Yes]
 [X3 = Normal]
  [X1 = Sunny]
   [Yes]
   [No]
  [X4 = Weak]
   [Yes]
   [Yes]


In [9]:
import numpy as np
import pandas as pd

def importData(path='tennis.csv'):
    data = pd.read_csv(path)
    return data

def entropy(data):
    target_values = data.iloc[:, -1].value_counts(normalize=True)
    entropy_val = -sum(target_values * np.log2(target_values))
    return entropy_val

def information_gain_ratio(data, attribute):
    # Calculate information gain
    entropy_before_split = entropy(data)
    values, counts = np.unique(data[attribute], return_counts=True)
    weighted_entropy_after_split = sum([(counts[i] / len(data)) * entropy(data[data[attribute] == values[i]]) for i in range(len(values))])
    information_gain = entropy_before_split - weighted_entropy_after_split

    # Calculate split information
    split_info = -sum((counts[i] / len(data)) * np.log2(counts[i] / len(data)) for i in range(len(values)))

    # Calculate information gain ratio
    if split_info == 0:
        return 0  # To avoid division by zero
    information_gain_ratio_val = information_gain / split_info

    return information_gain_ratio_val

def choose_attribute(data, attributes):
    information_gain_ratios = [information_gain_ratio(data, attribute) for attribute in attributes]
    best_attribute_index = np.argmax(information_gain_ratios)
    return attributes[best_attribute_index]

def split_data(data, attribute, value):
    return data[data[attribute] == value]

def majority_class(data):
    return data.iloc[:, -1].mode()[0]

def build_tree(data, attributes):
    # If all instances have the same class label, return a leaf node with that class label
    if len(np.unique(data.iloc[:, -1])) == 1:
        return data.iloc[0, -1]

    # If there are no more attributes to split on, return the majority class label
    if len(attributes) == 0:
        return majority_class(data)

    # Choose the best attribute to split on
    best_attribute = choose_attribute(data, attributes)

    # Create a new decision tree node with the best attribute
    tree = {best_attribute: {}}

    # Remove the best attribute from the list of attributes
    attributes = [attr for attr in attributes if attr != best_attribute]

    # Split the dataset based on the values of the best attribute
    for value in np.unique(data[best_attribute]):
        sub_data = split_data(data, best_attribute, value)
        subtree = build_tree(sub_data, attributes)
        tree[best_attribute][value] = subtree

    return tree

def print_tree(tree, depth=0):
    if isinstance(tree, dict):
        for attribute, subtree in tree.items():
            print('  ' * depth + attribute)
            for value, sub_tree in subtree.items():
                print('  ' * (depth+1) + '{} = {}'.format(attribute, value))
                print_tree(sub_tree, depth + 2)
    else:
        print('  ' * depth + '  [Class: ' + str(tree) + ']')

# Load the dataset
data = importData('tennis.csv')

# Get list of attribute names
attributes = list(data.columns[:-1])

# Build the decision tree using C4.5 algorithm
tree = build_tree(data, attributes)

# Print the decision tree
print_tree(tree)


Outlook
  Outlook = Overcast
      [Class: Yes]
  Outlook = Rain
    Wind
      Wind = Strong
          [Class: No]
      Wind = Weak
          [Class: Yes]
  Outlook = Sunny
    Humidity
      Humidity = High
          [Class: No]
      Humidity = Normal
          [Class: Yes]


In [10]:
# Get list of attribute names

import pandas as pd
# 
# Define the data
data = {
    'a1': ['x', 'x', 'x', 'y', 'y', 'x', 'x', 'y', 'x', 'x', 'y', 'x'],
    'a2': ['u', 'u', 'u', 'u', 'v', 'v', 'u', 'v', 'u', 'w', 'w', 'w'],
    'a3': ['n', 'p', 'n', 'n', 'n', 'n', 'p', 'm', 'n', 'p', 'n', 'n'],
    'a4': ['e', 'f', 'g', 'e', 'f', 'e', 'e', 'f', 'f', 'f', 'f', 'g'],
    'Class': ['+', '+', '+', '+', '-', '+', '-', '+', '+', '+', '-', '+']
}

# Create DataFrame
data = pd.DataFrame(data)

attributes = list(data.columns[:-1])

# Build the decision tree using C4.5 algorithm
tree = build_tree(data, attributes)

# Print the decision tree
print_tree(tree)

a1
  a1 = x
    a3
      a3 = n
          [Class: +]
      a3 = p
        a4
          a4 = e
              [Class: -]
          a4 = f
              [Class: +]
  a1 = y
    a3
      a3 = m
          [Class: +]
      a3 = n
        a4
          a4 = e
              [Class: +]
          a4 = f
              [Class: -]


In [11]:
test = {'a1': 'x', 'a2': 'u', 'a3': 'n', 'a4': 'e'}


In [12]:
# Get list of attribute names

import pandas as pd

data = {
    'Day': [f'D{i}' for i in range(1, 15)],
    'Outlook': ['Sunny', 'Sunny', 'Overcast', 'Rain', 'Rain', 'Rain', 'Overcast', 'Sunny', 'Sunny', 'Rain', 'Sunny', 'Overcast', 'Overcast', 'Rain'],
    'Temperature (oF)': [85, 80, 83, 70, 68, 65, 64, 72, 69, 75, 75, 72, 81, 71],
    'Humidity (%)': [85, 90, 86, 96, 80, 70, 65, 95, 70, 80, 70, 90, 75, 91],
    'Wind': ['Weak', 'Strong', 'Weak', 'Weak', 'Weak', 'Strong', 'Strong', 'Weak', 'Weak', 'Weak', 'Strong', 'Strong', 'Weak', 'Strong'],
    'PlayTennis': ['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No']
}
# Create DataFrame
data = pd.DataFrame(data)

attributes = list(data.columns[:-1])

# Build the decision tree using C4.5 algorithm
tree = build_tree(data, attributes)

# Print the decision tree
print_tree(tree)

Day
  Day = D1
      [Class: No]
  Day = D10
      [Class: Yes]
  Day = D11
      [Class: Yes]
  Day = D12
      [Class: Yes]
  Day = D13
      [Class: Yes]
  Day = D14
      [Class: No]
  Day = D2
      [Class: No]
  Day = D3
      [Class: Yes]
  Day = D4
      [Class: Yes]
  Day = D5
      [Class: Yes]
  Day = D6
      [Class: No]
  Day = D7
      [Class: Yes]
  Day = D8
      [Class: No]
  Day = D9
      [Class: Yes]


In [14]:
import numpy as np
import pandas as pd

def importData(path='tennis.csv'):
    data = pd.read_csv(path)
    return data

def entropy(data):
    target_values = data.iloc[:, -1].value_counts(normalize=True)
    entropy_val = -sum(target_values * np.log2(target_values))
    return entropy_val

def information_gain_ratio(data, attribute):
    # Calculate information gain
    entropy_before_split = entropy(data)
    values, counts = np.unique(data[attribute], return_counts=True)
    weighted_entropy_after_split = sum([(counts[i] / len(data)) * entropy(data[data[attribute] == values[i]]) for i in range(len(values))])
    information_gain = entropy_before_split - weighted_entropy_after_split

    # Calculate split information
    split_info = -sum((counts[i] / len(data)) * np.log2(counts[i] / len(data)) for i in range(len(values)))

    # Calculate information gain ratio
    if split_info == 0:
        return 0  # To avoid division by zero
    information_gain_ratio_val = information_gain / split_info

    return information_gain_ratio_val

def choose_attribute(data, attributes):
    information_gain_ratios = [information_gain_ratio(data, attribute) for attribute in attributes]
    best_attribute_index = np.argmax(information_gain_ratios)
    return attributes[best_attribute_index]

def split_data(data, attribute, value):
    return data[data[attribute] == value]

def majority_class(data):
    return data.iloc[:, -1].mode()[0]

def build_tree(data, attributes):
    # If all instances have the same class label, return a leaf node with that class label
    if len(np.unique(data.iloc[:, -1])) == 1:
        return data.iloc[0, -1]

    # If there are no more attributes to split on, return the majority class label
    if len(attributes) == 0:
        return majority_class(data)

    # Choose the best attribute to split on
    best_attribute = choose_attribute(data, attributes)

    # Create a new decision tree node with the best attribute
    tree = {best_attribute: {}}

    # Remove the best attribute from the list of attributes
    attributes = [attr for attr in attributes if attr != best_attribute]

    # Split the dataset based on the values of the best attribute
    for value in np.unique(data[best_attribute]):
        sub_data = split_data(data, best_attribute, value)
        subtree = build_tree(sub_data, attributes)
        tree[best_attribute][value] = subtree

    return tree

def print_tree(tree, depth=0):
    if isinstance(tree, dict):
        for attribute, subtree in tree.items():
            print('  ' * depth + attribute)
            for value, sub_tree in subtree.items():
                print('  ' * (depth+1) + '{} = {}'.format(attribute, value))
                print_tree(sub_tree, depth + 2)
    else:
        print('  ' * depth + '  [Class: ' + str(tree) + ']')

def predict(tree, instance):
    # Recursively traverse the tree to make predictions
    for attribute, subtree in tree.items():
        value = instance[attribute]
        if value in subtree:
            # If the attribute value exists in the subtree, traverse deeper
            if isinstance(subtree[value], dict):
                return predict(subtree[value], instance)
            else:
                return subtree[value]  # Return the predicted class label
        else:
            # If the attribute value is not found in the subtree, return the majority class
            return majority_class(data)

def evaluate_predictions(data, tree):
    predicted_labels = [predict(tree, instance) for _, instance in data.iterrows()]
    actual_labels = data.iloc[:, -1]
    return actual_labels, predicted_labels

def classification_report(actual_labels, predicted_labels):
    true_positives = sum(1 for a, p in zip(actual_labels, predicted_labels) if a == p)
    false_positives = sum(1 for a, p in zip(actual_labels, predicted_labels) if a != p)
    accuracy = true_positives / len(actual_labels)
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / len(actual_labels)
    f1_score = 2 * (precision * recall) / (precision + recall)
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score
    }

# Load the dataset
data = importData('tennis.csv')

# Get list of attribute names
attributes = list(data.columns[:-1])

# Build the decision tree using C4.5 algorithm
tree = build_tree(data, attributes)

# Print the decision tree
print_tree(tree)

# Evaluate the model
actual_labels, predicted_labels = evaluate_predictions(data, tree)

# Compute classification report
report = classification_report(actual_labels, predicted_labels)
from sklearn.metrics import confusion_matrix
# Print classification report
print("\nClassification Report:")
print("Accuracy:", report['accuracy'])
print("Precision:", report['precision'])
print("Recall:", report['recall'])
print("F1 Score:", report['f1_score'])
# Print confusion matrix
conf_matrix = confusion_matrix(actual_labels, predicted_labels)

print("\nConfusion Matrix:")
print(conf_matrix)

Outlook
  Outlook = Overcast
      [Class: Yes]
  Outlook = Rain
    Wind
      Wind = Strong
          [Class: No]
      Wind = Weak
          [Class: Yes]
  Outlook = Sunny
    Humidity
      Humidity = High
          [Class: No]
      Humidity = Normal
          [Class: Yes]

Classification Report:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

Confusion Matrix:
[[5 0]
 [0 9]]
